In [1]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import display, Image
from langgraph.graph import START, END, MessagesState, StateGraph
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults
import warnings
warnings.filterwarnings("ignore")

In [2]:
load_dotenv()

class Application:
    def __init__(self):
        self.model = ChatOpenAI(model="gpt-4o", api_key=os.getenv("api_key"), temperature=0)
        self.browser = DuckDuckGoSearchResults(name="duckduckgo_search",
                                  description="Search DuckDuckgo and return structured results")
        self.tools = self.create_tools()
        self.model_with_tools = self.model.bind_tools(self.tools)
        self.memory = MemorySaver()
        self.graph = self.build_graph()
        
    def create_tools(self):
        @tool
        def get_weather(city: str) -> str:
            """
            Returns current weather for a given city using WeatherAPI.com
            Usage: get_weather("Lagos")
            """
            weather_api = os.getenv("weather_api_key")
            endpoint = f"https://api.weatherapi.com/v1/current.json?key={weather_api}&q={city}"

            try:
                res = requests.get(endpoint, timeout=5).json()

                location = res["location"]["name"]
                country  = res["location"]["country"]
                temp     = res["current"]["temp_c"]
                condition = res["current"]["condition"]["text"]
                humidity = res["current"]["humidity"]
                wind = res["current"]["wind_kph"]

                return (
                    f"Weather in {location}, {country}:\n"
                    f"Condition: {condition}\n"
                    f"Temperature: {temp}°C\n"
                    f"Humidity: {humidity}%\n"
                    f"Wind: {wind} kph\n"
                )

            except Exception as e:
                return f"Failed to fetch weather. Error: {e}"
        
        SCIENTIFIC_DICTIONARY = {
        "neuron": "A neuron is a specialized cell in the nervous system that transmits information through electrical and chemical signals.",
        "entropy": "Entropy is a measure of disorder or randomness in a system, commonly used in thermodynamics and information theory.",
        "algorithm": "An algorithm is a finite sequence of well-defined steps used to solve a problem or perform a computation.",
        "gradient descent": "Gradient descent is an optimization algorithm used to minimize a function by iteratively moving in the direction of steepest descent.",
        "quantum": "Quantum refers to the smallest discrete unit of a physical property, central to quantum mechanics.",
        "relativity": "Relativity is a theory by Albert Einstein describing the relationship between space, time, and gravity.",
        "neural network": "A neural network is a computational model inspired by the human brain, consisting of interconnected nodes that process information.",
        "backpropagation": "Backpropagation is an algorithm for training neural networks by computing gradients of the loss function."}

        @tool
        def dictionary_lookup(term: str) -> str:
            """
            Look up definitions of tax-related terms and acronyms.
            """
            key = term.lower().strip()
            return SCIENTIFIC_DICTIONARY.get(
                key,
                f"No definition found for '{term}'. Try a different term."
            )
    
        return [get_weather, dictionary_lookup, self.browser]
    
    def build_graph(self):

        #create node to access llm
        def access_model(state:MessagesState) -> MessagesState:
            """
            This node is responsible for making calls to openai llm
            """
            sys_message = """
                            You are an AI assistant, your goal is to answer questions asked by users to the best
                            of your abilities. Tools have been provided for you to refer to help answer queries.
                        """
            response = self.model_with_tools.invoke([SystemMessage(sys_message)] + state["messages"])
            return {"messages": [response]}

        def should_continue(state:MessagesState):
            if state["messages"][-1].tool_calls:
                return "tools"
            else:
                return "exit"
        
        #Build graph

        #create tool node
        tools = self.tools
        tool_node = ToolNode(tools=tools)
        memory = self.memory

        graph  = StateGraph(MessagesState)
        graph.add_node(node = "access_model", action=access_model)
        graph.add_node(node = "tool_node", action = tool_node)
        graph.add_edge(START, "access_model")
        graph.add_conditional_edges("access_model", should_continue,
                                    {"tools": "tool_node",
                                    "exit": END})
        graph.add_edge("tool_node", "access_model")
        app = graph.compile(checkpointer=memory)
        return app
    
    def visualize_graph(self):
        return display(Image(self.graph.get_graph().draw_mermaid_png()))
    
    def query_agent(self, query: str, thread_id: str = "user_001"):
        app = self.graph
        response = app.invoke({"messages": [HumanMessage(content=query)]}, 
                            config={"configurable": {"thread_id": thread_id}})
        latest_response = response["messages"][-1]
        return f"You: {query} \nAI: {latest_response.content}"

In [ ]:
queries = [
    "Define an algorithm for me", #Test dictionary tool
    "What is the weather going to be like tomorrow in Lagos", #Test weather tool
    "Explain Integrals to me", #Random question without the need for tool
    "Can you explain what is going on in Venezuela"
]
apps = Application()

def test_model():
    for query in queries:
        print("\nQuery\n")
        print(apps.query_agent(query))
        
test_model()